텍스트 생성 모델 1

5점을 긍정으로 보고 나머지는 부정으로 보는 모델 2

1모델에서 나온 문장을 2모델에 삽입

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, Dropout,Input
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from tqdm import tqdm
import json

In [ ]:
data= pd.read_csv('h:/내 드라이브/알파코 수업자료 원본/프로젝트2/자료+욕설.csv',encoding='cp949')
# with open('/content/data.pickle','rb') as f:
#     data= pickle.load(f)
data

In [ ]:
df=data[['Text','Score']]
df

In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

df

In [5]:
X=df['Text'].values
y=df['Score'].values


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)


In [7]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
nums = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(nums)) # 전체 단어 개수 확인
num_word=len(nums)+1

전체 단어 개수:  134704


In [8]:
maxlen=15
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='pre', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='pre', truncating='post')


In [ ]:
input_layer = Input(shape=(maxlen,))

# 임베딩 레이어 추가
embedding_layer = Embedding(input_dim=num_word, output_dim=32, input_length=maxlen)(input_layer)

# LSTM 레이어 추가
lstm_layer = LSTM(1024,return_sequences=True)(embedding_layer)
lstm_layer = LSTM(2048,return_sequences=True)(embedding_layer)
lstm_layer = LSTM(512,return_sequences=True)(embedding_layer)
lstm_layer = LSTM(64)(embedding_layer)
# 출력 레이어 추가
output_layer = Dense(1, activation='sigmoid')(lstm_layer)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 훈련

model.fit(X_train_pad, y_train, epochs=20, validation_data=(X_test_pad, y_test), callbacks=[early_stopping],verbose=1)




Epoch 1/20
2280/2280 [==============================] - 12s 5ms/step - loss: 0.1180 - accuracy: 0.9526 - val_loss: 0.0602 - val_accuracy: 0.9771
Epoch 2/20
2280/2280 [==============================] - 11s 5ms/step - loss: 0.0161 - accuracy: 0.9945 - val_loss: 0.0660 - val_accuracy: 0.9699
Epoch 3/20
2280/2280 [==============================] - 11s 5ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.1436 - val_accuracy: 0.9548
Epoch 4/20
2280/2280 [==============================] - 11s 5ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.2082 - val_accuracy: 0.9453


In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

  1/570 [..............................] - ETA: 5s - loss: 0.1191 - accuracy: 0.9375

570/570 [==============================] - 1s 2ms/step - loss: 0.0602 - accuracy: 0.9771
Accuracy: 97.71%


In [ ]:
real_text='미친거 아니야!'
def blackconsumer(real_text):
    new_sentence=tokenizer.texts_to_sequences(real_text)

    flat_list = [item for sublist in new_sentence for item in sublist]

    X_real_pad = pad_sequences([flat_list] ,maxlen=maxlen, padding='pre')

    score = float(model.predict(X_real_pad, verbose=0))
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
        
blackconsumer(real_text)     


95.75% 확률로 부정 리뷰입니다.



In [ ]:
num=6000
ds=data[['Text']][num:num+30]

for i in ds['Text']:
    print(i)
    blackconsumer(i)

아 네 안내 도와드리겠습니다. 선생님 혹시 차량 번호 불러주시겠습니까?
76.68% 확률로 부정 리뷰입니다.

육십팔 노 일공일일 고 법인차량이거든요
100.00% 확률로 긍정 리뷰입니다.

일공일일번 차량이구요 지금 한 번 확인을 해 볼 텐데 방문하실 때는 양수 받으시는 개인 분이랑 법인 쪽에서도 사람이 나오시는 거 맞죠 선생님
97.08% 확률로 부정 리뷰입니다.

예
99.83% 확률로 긍정 리뷰입니다.

음 혹시 뭐 세금계산서라든지 이런 거는
99.36% 확률로 긍정 리뷰입니다.

제가 지금 법인 대표인데 법인을 인자 해지를 해야 되기 때문에 제 앞으로 이전을 할려고 하거든요 개인으로 이런 이런 경우엔 제가 제가 혼자가서 다 하면 되는 거죠
98.14% 확률로 긍정 리뷰입니다.

고 차량이전 할려면은 그 서류 혹시 필요해요
100.00% 확률로 긍정 리뷰입니다.

네 자동차등록증 예 그리고 두 분 각각 신분증예 그리고 차량 주행거리 확인을 해야 되고 그리고 현재 번호판이 대구 지역번호판이다 보니까 번호랑 번호판을 교체를 같이 하셔야 합니다. 그래서 차 가지고 방문을 해 주시면 되겠습니다.
99.09% 확률로 긍정 리뷰입니다.

아 예 그러면 선생님 제가 정확하게 확인 위해서 차량 번호 확인가능하십니까?
80.32% 확률로 부정 리뷰입니다.

대구 삼십마 육사오일요
100.00% 확률로 긍정 리뷰입니다.

아 예 선생님 기존의 차주님 성함이 어떻게 됩니까?
99.65% 확률로 부정 리뷰입니다.

이선옥이요
99.97% 확률로 긍정 리뷰입니다.

네 확인 감사합니다. 그러면 선생님 명의로 이전 받는 부분이 맞습니까?
99.87% 확률로 부정 리뷰입니다.

예
99.83% 확률로 긍정 리뷰입니다.

아 예 그럼 방문하시는 분에 따라서 서류가 달라지는데 어느분이 방문하시는거에요
99.98% 확률로 긍정 리뷰입니다.

아 번호판도 교체해야 돼요
95.12% 확률로 긍정 리뷰입니다.

카면 신분증만 있으면 돼요 두 두 명 다
99.96% 확률로 긍정 리뷰입니다.


In [ ]:
ds

,Text
5000,그러면 선생님 신차에 그 제 자동차 제작증상에 공동명의로 되어있으신가요
5001,예예
5002,아 그러시다면 그 명의 그대로 공동명의로 가능한 부분인데요 혹시 장애인 해당 되시나요
5003,그런건 아니고예
5004,이거나 엘피지 차량인가요 선생님
5005,그런건 아니고예
5006,그런 부분은 아니시구요 그럼 필요한 서류는 제가 확인을 해드리도록 하겠습니다 혹시 ...
5007,어 뭐 차량 이전 때문에 안내 좀 받을라 카거든요
5008,예 그러면 주소이전은 등기부 등본상에 주소지 변경이 있으면 반드시 방문해서 신고하셔...
5009,법인에서 법인입니까?


In [ ]:
model.save('긍부정모델_완.keras')